# Ensemble Evaluation: Timepoint 3

Location: New York State

Timepoint 3: January 4, 2022. Setting: New York State coinciding with the arrival of the first Omicron wave. At-home testing widely available.

For each timepoint, consider the following:
 - What is the most relevant data to use for model calibration?
 - What was our understanding of COVID-19 viral mechanisms at the time? For example, early in the pandemic, we didn't know if reinfection was a common occurance, or even possible.
 - What are the parameters related to contagiousness/transmissibility and severity of the dominant strain at the time?
 - What policies were in place for a stated location, and how can this information be incorporated into models?

For each timepoint:

1. (a) Take a single model, calibrate it using any historical data prior to the given date, and create a 4-week forecast for cases, hospitalizations, and deaths beginning on the given date. (b) Evaluate the forecast using the COVID-19 Forecasting Hub Error Metrics (WIS, MAE). The single model evaluation should be done in the same way as the ensemble.

2. Repeat (1), but with an ensemble of different models.

a. It is fine to calibrate each model independently and weight naively.

b. It would also be fine to calibrate the ensemble as a whole, assigning weights to the different component models, so that you minimize the error of the ensemble vs. historical data.

c. Use the calibration scores and error metrics computed by the CDC Forecasting Hub. As stated on their website:

“Periodically, we evaluate the accuracy and precision of the ensemble forecast and component models over recent and historical forecasting periods. Models forecasting incident hospitalizations at a national and state level are evaluated using adjusted relative weighted interval scores (WIS, a measure of distributional accuracy), and adjusted relative mean absolute error (MAE), and calibration scores. Scores are evaluated across weeks, locations, and targets. You can read a paper explaining these procedures in more detail, and look at the most recent monthly evaluation reports. The final report that includes case and death forecast evaluations is 2023-03-13.”

3. Produce the forecast outputs in the format specified by the CDC forecasting challenge, including the specified quantiles.

### Load dependencies

In [1]:
import os
import pandas as pd
import numpy as np
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_and_optimize_and_sample_petri_model,
    load_and_calibrate_and_optimize_and_sample_petri_model
)
from pyciemss.visuals import plots

## Get data

In [7]:
url = 'https://raw.githubusercontent.com/DARPA-ASKEM/experiments/main/thin-thread-examples/milestone_12month/evaluation/ensemble_eval_SA/datasets/aabb3684-a7ea-4f60-98f1-a8e673ad6df5/dataset.csv'
ny_data = pd.read_csv(url)

# Grab test data for four-week forecast (01/04/2022 - 02/01/2022)
test_data = ny_data[0:742].drop(columns="timestep")

# Select historical data up to Timepoint 3: 01/03/2022 (the first 714 rows)
ny_data = ny_data[0:713]
ny_data[["I", "H", "D"]].to_csv("NY_data3.csv")

## Select relevant models